# Assignment Machine Learning
***Assignment BIT07 / Bioinformatics @ home***

**Student name**: Glenn Broeckx  
**Student number**: 202054838

## PART 1: Choosing the dataset
For this assignment, a dataset on breast cancer characteristics, RNA expression and survival was chosen. This dataset is originating from the METABRIC clinical trials published in Nature (2012), wherein an European research group mapped genomic and transcriptomic profiles in breast cancer. (1)

This dataset has a mixture of categorical variables (both ordinal and nominal) and numerical variables. It has the possibility to predict survival in both status as survival duration. Furhtermore, it is possible to predict the type of breast cancer based on gene expression profiling and gene mutational profiling, which was the main goal of the METABRIC clinical trial. 

The METABRIC dataset is freely available on the Kaggle website under the Database Contents License (DbCL) [v1.0] from following url: [https://www.kaggle.com/raghadalharbi/breast-cancer-gene-expression-profiles-metabric](https://www.kaggle.com/raghadalharbi/breast-cancer-gene-expression-profiles-metabric).

In this Jupyter Notebook file, all modules, methods and functions will be imported. Several own created functions will be declared as well.

In [2]:
# Importing modules, functions and methods
###################################################
# Data analysis and scientific calculations
from scipy import stats
import numpy as np
import pandas as pd
import opendatasets as od

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
rcParams['figure.figsize'] = 20,8.27 #size of figures in inch

# Machine learning modules
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc, det_curve, balanced_accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer


# Own functies
################
def listdiff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif


def ohe_and_bind(original_dataframe, features_to_encode):
    if type(features_to_encode) is not list:
        raise Exception("Features to encode needs to be a list")

    for f_encode in features_to_encode:
        dummies = pd.get_dummies(original_dataframe[[f_encode]])
        original_dataframe = original_dataframe.drop(f_encode, axis=1)
        original_dataframe = pd.concat([original_dataframe, dummies], axis=1)

    return(original_dataframe)

## PART 2: Describing the dataset
The downloaded dataset is imported into a Pandas DataFrame, called `bc`, short for breast cancer. In this Notebook, variables derived from `bc` will further be named with a `bc_` prefix, followed by a short indication of derivation. 

In [4]:
# 2.1: Importing dataset
bc = pd.read_csv('METABRIC_RNA_Mutation.csv')
bc.head()

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (678,688,690,692) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
0,0,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0,claudin-low,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
1,2,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
2,5,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
3,6,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
4,8,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0


In [18]:
bc_shape = bc.shape
print(f'Number of rows/observations/patients in the dataframe:\t{bc_shape[0]:5d}')
print(f'Number of columns/features/attributes in the dataframe:\t{bc_shape[1]:5d}')

Number of rows/observations/patients in the dataframe:	 1904
Number of columns/features/attributes in the dataframe:	  693


The `head()` method defaulty shows the first 5 rows of the dataframe. It is followed with the dimensions of the shown dataframe, But because only the head method is used, the number of observations/rows was limited to 5. The `shape` method gives a better overview of the overall dimensions of the dataframe. As presented in the output of previous code cell, these are the dimensions:

- Number of rows/observations/patients in the dataframe: 1904
- Number of columns/features/attributes in the dataframe: 693

## 2.1 Describing the attributes
As previously discussed, 693 attributes are present in this dataset. These are quite a lot to individually discuss them in a table. The high number of attributes is caused by the genomic and transcriptomic mapping, in which each gene has its own attribute. Transcriptomic mapping starts at column with name `brca1`, genomics mapping starts at column with name `pik3ca_mut`. In the next cell, the indices of thise columns are given.

In [16]:
print(f'Starting index of transcriptomics (starting at brca1:\t{bc.columns.get_loc("brca1"):5d}')
print(f'Starting index of genomics (starting at pik3ca_mut:\t{bc.columns.get_loc("pik3ca_mut"):5d}')

Starting index of transcriptomics (starting at brca1:	   31
Starting index of genomics (starting at pik3ca_mut:	  520


In following overview, all attributes concerning transcriptomics (columns with index 31 to 519) and all attributes concerning genomics (columns with index 520 to 692) are lumped together for readability.

The following overview is based on information from the Kaggle website and from the METABRIC publication.

In [ ]:
# Stap 1.2: beschrijven van de dataset
df.shape #dimensies van de dataset
df.describe() # range van waarden; vul ook aan met info over kolommen van bron

In [ ]:
# Stap 1.3: Zoeken naar missing values
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Stap 1.4: Zoeken naar inconsistente
df.describe() # range van waarden kloppen met mogelijke werkelijkheid? Te veel nullen als missing values?

In [ ]:
# Stap 1.5: Zoeken naar uitschieters
columns = df.columns()

for column in columns:
    ax1 = sns.boxplot(x=column, data=df) #interessanter voor categorische data
    ax2 = sns.histplot(x=column, data=df) #beter voor numerieke data

In [ ]:
# Stap 1.6: Verwijderen van data
df.drop('column', axis=1, inplace=True) #kolom verwijderen wegens niet contributief of missing values
df.drop(df[(df.column1 == 0) | (df.column2 == 0) | (df.column3 == 0)].index, inplace = True) #Rijen verwijderen met foutieve waarde 0
df = df[(np.abs(stats.zscore(df)) < 5).all(axis=1)] #Verwijderen van uitschieters die meer dan 5x SD verwijderd zijn van gemiddelde
df = df.dropna() #Verwijderen van rijen met missing values

### Conclusie stap 1
Deze dataset bestaat uit volgende kolommen (+info):
- Kolom 1: info
- Kolom 2: info
- ...

Er zijn X observaties en Y kolommen. Hierin zijn er A vreemde waarden/missing values. Deze worden verwijderd/behouden/omvormd. Omvormen (imputing) kan pas na het splitsen van de dataset.

## Stap 2: Analyse en preprocessing
**Stap 2.1: analyse**  
Bij de analytische fase is het de bedoeling om te kijken naar welke features een invloed hebben op het target. Dit kan op volgende manieren:
- Pairplot (moeilijk leesbaar bij veel features --> exporteren en zoomen)
- Heatmap (moeilijk leesbaar bij veel features --> exporteren en zoomen)
- Afzonderlijke scatterplots
- Boxplots van categorieke features
- Gebalanceerdheid van de data

Creativiteit de vrije loop laten gaan!

**Stap 2.2 preprocessing**  
Voor het preprocessen zijn er verschillende stappen te ondergaan:
1. Omvormen van ordinale categorieën (ordinal encoding)
2. Omvormen van categorische waarden (one-hot encoding)
3. Opsplitsen in features en targets
4. Opsplitsen in testset en trainingsset
4. Omvormen van missing values (imputing)
5. Schalen van numerieke features

Bij elke stap laten weten op welke manier en waarom je dit doet.

In [ ]:
# Stap 2.1.1: Pairplot
sns.pairplot(df)

In [ ]:
# Stap 2.1.2 Heatmap
df_corr = df.corr()
f, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(df_corr, 
            mask=np.zeros_like(df_corr, dtype=bool), 
            cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, 
            ax=ax,annot=True
           )

In [ ]:
# Stap 2.1.3: Afzonderlijke scatterplots
sns.scatterplot(data=df, x='feature', y='target')

In [ ]:
# Stap 2.1.4: Boxplots van categorieke waarden
sns.boxplot(x=df["feature"], y=df["target"])

In [ ]:
# Stap 2.1.5: Gebalanceerdheid van de data op basis van target
df['target'].value_counts() #absolute waarden
df['target'].value_counts(normalize=True) #percentages
sns.countplot(x="target", data=df) #histogram

### Conclusie stap 2.1
Formuleer enkele bevindingen van de data-analyse. Kopieer deze ook al naar de eindconclusie.

In [ ]:
# Stap 2.2.1: Omvormen ordinale categorieën: rankings naar nummers
cat1 = ['rank1', 'rank2', 'rank3'] # categoriesleutel op basis van de vooranalyse voor feature 1
cat2 = ['rank1', 'rank2', 'rank3'] # categoriesleutel op basis van de vooranalyse voor feature 2

ord_features = ['feature1', 'feature2'] # lijst met alle colomnamen met ordinale features
rest_features = listdiff(df.columns, ord_features)

ordinal_trf = ColumnTransformer(
    transformers =[
        ('ord', #gewoon een naam
         preprocessing.OrdinalEncoder(categories=[cat1, cat2]), # Belangrijk!: er moeten evenveel categorieën zijn als aantal om te zetten features. Deze moeten in volgorde staan en als twee features zelfde categorieën hebben mag dit herhaald worden
         ord_features), #Lijst met de features om te transformeren
], remainder ='passthrough')

df = pd.DataFrame(ordinal_trf.fit_transform(df), columns=[ord_features, rest_features])

In [ ]:
# Stap 2.2.2: One-hot encoding
ohe_features = [featureA, featureB]
df = ohe_and_bind(df, ohe_features)

In [ ]:
# Stap 2.2.3: Opsplitsen in features en target
y = df['target'].values.astype('int')
X = pd.DataFrame(df.drop('target', axis=1))

In [ ]:
# Stap 2.2.4: Opsplitsen in training en testset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, random_state=0)

In [ ]:
# Stap 2.2.5: Imputing (kan wegvallen indien niet nodig)
imp = SimpleImputer(missing_values=np.nan, strategy='mean') # of andere strategie
imp.fit(X_train)

imp.transform(X_train)
X_train = pd.DataFram(X_train, columns=imp.feature_names_in_)
imp.transform(X_test)
X_test = pd.DataFram(X_test, columns=imp.feature_names_in_)

In [ ]:
# Stap 2.2.6: Schaling (moet gebeuren!)
scaler = StandardScaler() # of andere schaler
scaler.fit(X_train)
scaler.transform(X_train)
scaler.transform(X_test)

### Conclusie stap 2.2
Maak een conclusie over alle preprocessing eventueel met vermelding van het aantal features nu. Ook altijd vermelden waarom je voor welke techniek hebt gekozen.

## Stap 3: Regressie
Gebruik lineaire regressie om een conintue variabele target te voorspellen.

Stappenplan:
1. Initieel model zonder tweaking
2. Initieel model beoordelen (MSE, MAE en R²)
3. Model tunen
    1. Nieuwe features creëren
    2. Polynomiale expansie
    3. Regularisatie
    4. Hyperparameter tuning: Grid search/Random search
4. Model selecteren op basis van Grid search
5. Finaal model evalueren

Het is moeilijk om hier voorbeeldcode te geven. Dit moet bijna altijd worden aangepast aan de situatie.

## Stap 4: Classificatie
Gebruik Logistische regressie, SVM om een categorieke variabele target te voorspellen.

Stappenplan (zelfde als bij lineaire regressie, maar meerdere type modellen te testen bij Grid search):
1. Initieel model zonder tweaking
2. Initieel model beoordelen (Precisie, Recall, Accuracy, Confusiematrix, ROC en AUC)
3. Model tunen
    1. Nieuwe features creëren
    2. Polynomiale expansie
    3. Regularisatie
    4. Hyperparameter tuning: Grid search/Random search
4. Model selecteren op basis van Grid search
5. Finaal model evalueren

Het is moeilijk om hier voorbeeldcode te geven. Dit moet bijna altijd worden aangepast aan de situatie. Bij Classificatie is het altijd leuk om de ROC curves te tekenen.

## Stap 5: Clustering
Gebruik een clustering methode om een cluster te voorspellen.

Stappenplan (zelfde als bij lineaire regressie, maar meerdere type modellen te testen bij Grid search):
1. Initieel model zonder tweaking
2. Initieel model beoordelen (nog op te zoeken: evaluatieparameters)
3. Model tunen
    1. Nieuwe features creëren
    2. Polynomiale expansie
    3. Regularisatie
    4. Hyperparameter tuning: Grid search/Random search (? geen idee of dit past)
4. Model selecteren op basis van Grid search
5. Finaal model evalueren

Het is moeilijk om hier voorbeeldcode te geven. Dit moet bijna altijd worden aangepast aan de situatie. We moeten blijkbaar niet alle attributen gebruiken in het model. We mogen ons beperken tot de attributen met een goede performance of een combinatie van verschillende goede attributen. We moeten het wel proberen te baseren op dezelfde attributen als bij classificatie.

In [ ]:
# Stap x.1: Initieel model
model = module.Methode(parameters) # Aan te passen voor het type model je wilt gebruiken
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Voorbeelden modellen:
lregmodel = linear_model.LinearRegression()
ridgemodel = linear_model.Ridge(alpha=alpha,tol=0.0001,fit_intercept=True) # L2 regularisatie
lasso = linear_model.Lasso(alpha=alpha,tol=0.0001,fit_intercept=True) # L1 regularisatie
logrmodel = linear_model.LogisticRegression(C=1, solver='liblinear', class_weight='balanced', multi_class='ovr')
svmmodel = svm.SVC(kernel='linear',C=0.01)
# Wordt nog verder aangevuld

In [ ]:
# Stap x.2: Model beoordelen
## Linear, Ridge en Lasso
r2train = lregmodel.score(X_train, y_train)
r2test = lregmodel.score(X_test, y_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

## Logistic en SVM
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

## Clustering
# Wordt nog aangevuld

In [ ]:
# Stap x.3: Model tunen (werk opnieuw vanaf de basisset)
## x.3.1: Nieuwe features creëren
df.insert(df.columns.size-1, 'new_feature', df.feature1*df.feature2) # kan ook som, min, deling


## x.3.2: Polynomiale expansie
pol = df[['lijst met features met continue variabelen']]
nopol = lego.drop(['lijst met features met continue variabelen'], axis=1)

graad = 4 

poly = PolynomialFeatures(graad)
poly.fit(pol)
polynom = pd.DataFrame(poly.transform(pol), columns=poly.get_feature_names_out(pol.columns))

### Belangrijk! vul aan met One-hot encoding en ordinal encoding indien nodig, daarna opnieuw opsplitsen
ord_features = ['feature1', 'feature2'] # lijst met alle colomnamen met ordinale features
rest_features = listdiff(df.columns, ord_features)
nopol = pd.DataFrame(ordinal_trf.fit_transform(nopol), columns=[ord_features, rest_features])

ohe_features = [featureA, featureB]
nopol = ohe_and_bind(nopol, ohe_features)

df_preprocessed = pd.concat([polynom, nopol], axis=1)

y = df_preprocessed['target'].values.astype('int')
X = pd.DataFrame(df_preprocessed.drop('target', axis=1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, random_state=0)


## X.3.3: Regularisatie (kan ook tijdens gridsearch)
### Linear regression --> gebruik Lasso of Ridge regressie
### Logistic regression --> standaard is L2
lr = linear_model.LogisticRegression(C=1, solver='liblinear', class_weight='balanced', penalty='l1')
### SVM heeft geen regularisatie maar wel kernels
svmmodel = svm.SVC(kernel='poly',C=10000000)


## X.3.4: Hyperparameter tuning met grid search of random search
### linear regression alleen bij ridge/lasso en weinig zin
lrparam = {'alpha' = np.logspace(-5, 4, 10)}

grid_search = GridSearchCV(estimator = model, 
                           param_grid = lrparam,
                           scoring = 'r2', # roc_auc, f1_weighted, f1_macro, recall, ...
                           cv = 15,
                           n_jobs = -1,
                           verbose = 5)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  
y_pred = grid_search.predict(X_test)
r2train = lregmodel.score(X_train, y_train)
r2test = lregmodel.score(X_test, y_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)


### logistic regression grid search
logrparam = {
    'solver' = ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' = ['none', 'l1', 'l2', 'elasticnet'],
    'C' = np.logspace(-5, 4, 10)
}

grid_search = GridSearchCV(estimator = model, 
                           param_grid = logrparam,
                           scoring = 'accuracy', # roc_auc, f1_weighted, f1_macro, recall, ...
                           cv = 15,
                           n_jobs = -1,
                           verbose = 5)

grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  
y_pred = grid_search.predict(X_test)
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

### logistic regression random search
logrparam = {
    'solver' = ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' = ['none', 'l1', 'l2', 'elasticnet'],
    'C' = loguniform(1e-5, 100)
}

random_search = RandomizedSearchCV(model, 
                                   logrparam, 
                                   n_iter=20, 
                                   scoring='accuracy', 
                                   n_jobs=-1, 
                                   cv=5, 
                                   random_state=1)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  
y_pred = random_search.predict(X_test)
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

### SVM grid search
svmparamaters = [ 
        {'kernel': ['linear'], 'C': np.linspace(0.01,20,10)},
        {'kernel': ['rbf'], 'C': np.linspace(0.01,20,10), 'gamma': [0.0001, 0.001, 0.01, 0.1, 0.2]},
        {'kernel': ['poly'], 'C':np.linspace(0.01,20,10)} ]

grid_search = GridSearchCV(estimator = model, 
                           param_grid = svmparamaters,
                           scoring = 'accuracy', # roc_auc, f1_weighted, f1_macro, recall, ...
                           cv = 15,
                           n_jobs = -1,
                           verbose = 5)

grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_ 
best_parameters = grid_search.best_params_  
y_pred = grid_search.predict(X_test)
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

### SVM random search
model = SVC(probability=True)
parameters = {'kernel': ['linear','rbf','poly'],
              'C': uniform(0.01, 20), # haal C uit een random uniform distribution
              'gamma': uniform(0.001, 0.2)}


n_iter_search = 20

random_search = RandomizedSearchCV(model, param_distributions=parameters,cv=5,n_iter=n_iter_search,n_jobs = -1,verbose=1)

random_search = random_search.fit(X_train, y_train)

best_accuracy = random_search.best_score_ 
best_parameters = random_search.best_params_  
y_pred = random_search.predict(X_test)
report = classification_report(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)*100
matrix = confusion_matrix(y_test, y_pred)

#### Nog aan te vullen voor clustering

## Conclusion
Conclusie schrijven over:
- Dataset
    - Goed/slecht en waarom?
    - Problemen in de dataset
- Model
    - Beste model voor elke techniek
    - Performance
    - Overfitting/underfitting
    - Kan gebruikt worden in dagdagelijkse praktijk?

## References
1. Curtis, C., Shah, S.P., Chin, S.-F., Turashvili, G., Rueda, O.M., Dunning, M.J., Speed, D., Lynch, A.G., Samarajiwa, S., Yuan, Y., Gräf, S., Ha, G., Haffari, G., Bashashati, A., Russell, R., Mckinney, S., Langerød, A., Green, A., Provenzano, E., Wishart, G., Pinder, S., Watson, P., Markowetz, F., Murphy, L., Ellis, I., Purushotham, A., Børresen-Dale, A.-L., Brenton, J.D., Tavaré, S., Caldas, C., Aparicio, S., 2012. The genomic and transcriptomic architecture of 2,000 breast tumours reveals novel subgroups. Nature 486, 346–352.. doi:10.1038/nature10983